In [1]:
import glob
import os
import pandas as pd
import re
import numpy as np

def get_10_cv(file_path):
    pattern = "10-CV"
    lines = []
    with open(file_path, 'r') as f:
        # last_line = f.readlines()[-1]
        for line in f:
            if(re.search(pattern,line)):
                if(re.search("final",line)):
                    continue
                else:
                    lines.append(line)
    return lines

def get_result(file_path):
    lines = get_10_cv(file_path)
    # print(log)
    try:
        assert len(lines) == 32 or len(lines) == 1, len(lines)
    except:
        print(file_path, "not done")
    if(file_path == ""):
        print(lines)
    cvs, stds = [], []
    for line in lines:
        try:
            time,log_type,msg,cv_txt,std_txt,elapse = line.split('|')
            assert log_type == 'INFO', f"{line}"
            cv = float(cv_txt.split('=')[1])
            std = float(std_txt.split('=')[1])
            cvs.append(cv)
            stds.append(std)
        except Exception as e:
            print(file_path, line, e)
            raise e
    return round(np.array(cvs).mean(),5), round(np.array(stds).mean(),5)

In [2]:
path_files = glob.glob("output/*/*dependent*.log")
df = pd.DataFrame(columns=["subject_setup", "experimental_setup", "preprocess", "segment_lenght", "stimuli", "cv", "std"])
for path_file in path_files:
    # print(f)
    path, filename = os.path.split(path_file)
    # print(path, filename)
    filename, ext = os.path.splitext(filename)
    subject_setup, experimental_setup, stimuli,  preprocess, segment_lenght = filename.split('-')
    # print(subject_setup, experimental_setup, stimuli, preprocess, segment_lenght)
    try:
        cv, std = get_result(path_file)
        row = {
            "subject_setup": subject_setup,
            "experimental_setup": experimental_setup,
            "segment_lenght": segment_lenght,
            "stimuli": stimuli,
            "preprocess": preprocess,
            "cv": cv,
            "std": std
        }
        a = pd.DataFrame.from_records([row])
        df= pd.concat([df, a], axis=0)
    except Exception as e:
            # print(file_path, line, e)
            raise e
    # break
df.to_csv("output/ml.csv")

In [3]:
df = pd.read_csv("output/ml.csv")
# for preprocess in ["DE","DASM","RASM","DCAU","PCC_TIME","PCC_FREQ","PLV","PLI"]:
for subject_setup in ["dependent","independent"]:
    for experimental_setup in ["segment", 'trial']:
        for segment_lenght in [1,12,20,60]:
            for preprocess in ["DE","PCC_TIME"]:
                rows = df.loc[
                    (df["subject_setup"] == subject_setup) &
                    (df["experimental_setup"] ==  experimental_setup) &
                    (df["segment_lenght"] ==  segment_lenght) &
                    (df["preprocess"] ==  preprocess)
                ]
                if(len(rows) == 0): 
                    print(f"{subject_setup}-{experimental_setup}-sti-{preprocess}-{segment_lenght} has not run")
                elif(len(rows) == 1):
                    print(rows)
                    raise ValueError
                else:
                    print(f"{subject_setup}-{experimental_setup}-sti-{preprocess}-{segment_lenght}", round(rows['cv'].mean(),3), round(rows['std'].mean(),3), sep="\t")
                # pass
            # for sti in ["arousal","valence"]:


dependent-segment-sti-DE-1	0.685	0.026
dependent-segment-sti-PCC_TIME-1	0.671	0.026
dependent-segment-sti-DE-12	0.692	0.085
dependent-segment-sti-PCC_TIME-12	0.68	0.078
dependent-segment-sti-DE-20	0.684	0.107
dependent-segment-sti-PCC_TIME-20	0.666	0.11
dependent-segment-sti-DE-60	0.63	0.178
dependent-segment-sti-PCC_TIME-60	0.627	0.181
dependent-trial-sti-DE-1	0.634	0.101
dependent-trial-sti-PCC_TIME-1	0.62	0.094
dependent-trial-sti-DE-12	0.636	0.114
dependent-trial-sti-PCC_TIME-12	0.628	0.114
dependent-trial-sti-DE-20	0.646	0.124
dependent-trial-sti-PCC_TIME-20	0.63	0.128
dependent-trial-sti-DE-60	0.642	0.172
dependent-trial-sti-PCC_TIME-60	0.636	0.178
independent-segment-sti-DE-1	0.565	0.013
independent-segment-sti-PCC_TIME-1	0.547	0.021
independent-segment-sti-DE-12	0.516	0.07
independent-segment-sti-PCC_TIME-12	0.507	0.078
independent-segment-sti-DE-20	0.454	0.133
independent-segment-sti-PCC_TIME-20	0.431	0.091
independent-segment-sti-DE-60	0.409	0.13
independent-segment-sti-PCC_T